In [1]:
import nivapy3 as nivapy
import pandas as pd

In [2]:
eng = nivapy.da.connect()

Username:  ···
Password:  ··············


Connection successful.


# Maps for TEOTIL report

Rough notebook to generate data for the maps at the start of the annual TEOTIL reports.

## 1. Industry and sewage treatment

In [3]:
year = 2020

In [4]:
sql = ("""
select * from RESA2.RID_PUNKTKILDER
where anlegg_nr in (
select anlegg_nr from RESA2.RID_PUNKTKILDER_INPAR_VALUES
where year = %s
and inp_par_id in (26, 27, 34, 35, 44, 45))
""" % year)

df = pd.read_sql(sql, eng)
df.head()

,anlegg_nr,anlegg_navn,etat,opprettet,nedlagt,type,kno,adresse,postnr,regine,lon_utl,lat_utl,lon_anl,lat_anl,status
0,1149.0036.01,FMC BIOPOLYMER AS,SFT,None,None,INDUSTRI,1149,Vormedalsvegen 301,5545.0,039.80,5.318689,59.354188,5.318689,59.354188,Aktiv
1,1520.0008.01,Ørsta Stål AS,MVAMR,None,None,INDUSTRI,1520,Skorgeura,6150.0,095.210,6.043068,62.213521,6.043068,62.213521,Aktiv
2,1531.0011.01,Vedde Sildeoljefabrikk AS,MVAMR,None,None,INDUSTRI,1531,Brevika Industriv 4,6018.0,101.41,6.241718,62.436091,6.241718,62.436091,Aktiv
3,1535.0001.01,"TINE MIDT-NORGE BA, Tresfjord",MVAMR,None,None,INDUSTRI,1535,None,6391.0,102.61,7.128080,62.525529,7.128080,62.525529,Aktiv
4,1547.0005.01,Vikenco A.S,MVAMR,None,None,INDUSTRI,1547,Rindarøy,6480.0,106.430,6.921689,62.790761,6.921689,62.790761,Aktiv


In [5]:
ind_df = df.query("type == 'INDUSTRI'")
stan_df = df.query("type == 'RENSEANLEGG'")
print(len(ind_df))
print(len(stan_df))

156
2530


In [6]:
nivapy.spatial.quickmap(ind_df, lon_col="lon_utl", lat_col="lat_utl")

In [7]:
nivapy.spatial.quickmap(stan_df, lon_col="lon_utl", lat_col="lat_utl")

In [8]:
ind_df.to_csv(f"industry_{year}.csv")
stan_df.to_csv(f"large_sewage_treatment_{year}.csv")

## 2. Aquaculture

In [9]:
sql = ("""
select * from RESA2.RID_KILDER_AQUAKULTUR
where nr in (
select anlegg_nr from RESA2.RID_KILDER_AQKULT_VALUES
where ar = %s)
""" % year)

aq_df = pd.read_sql(sql, eng)
aq_df.head()

,nr,navn,etablert,type,komnr,plassering,miljo,kap,mh,midl,lengde,bredde,regine
0,11272,ØKSENGÅRD,2004.0,MIDLERTIDIG,1840,SJØ,SALTVANN,2700.0,TN,2012-12-31,67.137098,15.405270,163.31
1,11303,VEGGFJELL,1984.0,MIDLERTIDIG,1849,SJØ,SALTVANN,5400.0,TN,2012-12-31,67.960650,15.805867,170.22
2,11312,SVARTFJELL,1984.0,MIDLERTIDIG,1848,SJØ,SALTVANN,3600.0,TN,2012-12-31,67.939483,15.503168,169.52
3,11315,VINKFJORDEN,1983.0,MIDLERTIDIG,1848,SJØ,SALTVANN,3600.0,TN,2012-12-31,67.726378,15.379890,168.21
4,11320,OKSØY,1984.0,MIDLERTIDIG,1848,SJØ,SALTVANN,5400.0,TN,2012-12-31,67.974021,15.308233,169.7


In [10]:
print(len(aq_df))

885


In [11]:
nivapy.spatial.quickmap(aq_df, lon_col="bredde", lat_col="lengde")

In [12]:
aq_df.to_csv(f"aquaculture_{year}.csv")